In [1]:
import os

os.environ["LANGSMITH_API_KEY"] = "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"
os.environ["LANGSMITH_TRACING"] = "true"

In [2]:
from langchain.prompts import PromptTemplate
from langchain_groq import ChatGroq
from langchain.chains.router.llm_router import RouterOutputParser
from langchain_core.output_parsers import StrOutputParser

llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [3]:
# Primeira SequentialChain: Geração de nome de empresa e slogan

# Passo 1: Gerar nome de empresa com base na descrição
nome_empresa_prompt = PromptTemplate(
    input_variables=["input"],
    template="Com base na descrição: '{input}', sugira um nome para uma empresa."
)

# Passo 2: Gerar slogan com base no nome da empresa
slogan_empresa_prompt = PromptTemplate(
    input_variables=["nome"],
    template="Com base no nome da empresa '{nome}', sugira um slogan para uma campanha de propaganda."
)

nome_empresa_chain = nome_empresa_prompt | llm | StrOutputParser()
slogan_empresa_chain = slogan_empresa_prompt | llm | StrOutputParser()

empresa_chain = nome_empresa_chain | slogan_empresa_chain


In [4]:
# Segunda SequentialChain: Geração de nome de escola e slogan

# Passo 1: Gerar nome de escola com base na descrição
nome_escola_prompt = PromptTemplate(
    input_variables=["input"],
    template="Com base na descrição: '{input}', sugira um nome para uma escola."
)

# Passo 2: Gerar slogan com base no nome da escola
slogan_escola_prompt = PromptTemplate(
    input_variables=["nome"],
    template="Com base no nome da escola '{nome}', sugira um slogan para uma campanha de propaganda."
)

nome_escola_chain = nome_escola_prompt | llm | StrOutputParser()
slogan_escola_chain = slogan_escola_prompt | llm | StrOutputParser()

escola_chain = nome_escola_chain | slogan_escola_chain

In [5]:
default_prompt = PromptTemplate(
    input_variables=["input"],
    template="Dado um prompt a seguir ```{input}```, responda"
)

default_chain = default_prompt | llm | StrOutputParser()

In [6]:
MULTI_PROMPT_ROUTER_TEMPLATE = r"""
Dado um texto de entrada, seleciona o prompt que melhor se encaixa para a entrada.
Você será provido com um nome e uma descrição de situação em que o prompt deve ser utilizado.

<< Formatação >>
Retorne um código markdown com um objeto JSON formatado da seguinte forma:

```json
{{{{
    "destination": string \ nome do prompt usado ou "DEFAULT"
    "next_inputs": string \ o prompt de entrada
}}}}
```

<< PROMPTS CANDIDATOS >>
{destinations}

<< ENTRADA >>
{{input}}

<< OUTPUT (lembre de incluir o ```json)>>"""

destinations = """
'empresa: bom para responder sobre empresas\nescola: bom para responder sobre escolas'
"""

router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations
)

router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

In [7]:
destination_chains = {}
destination_chains['empresa'] = empresa_chain
destination_chains['escola'] = escola_chain

In [8]:
from operator import itemgetter
from typing import Literal
from typing_extensions import TypedDict
from langchain_core.runnables import RunnableLambda

def return_chain(x):
    
    if x["destination"] == "empresa":
        return(empresa_chain)
    
    if x['destination'] == "escola":
        return(escola_chain)
    
    return(default_chain)

class RouteQuery(TypedDict):
    """Route query to destination."""
    destination: Literal["empresa", "escola", "default"]

route_chain = (
    router_prompt
    | llm.with_structured_output(RouteQuery)
    | itemgetter("destination")
)

multi_prompt_chain = {
    "destination": route_chain,
    "input": lambda x: x["input"],
} | RunnableLambda(
    lambda x: return_chain(x),
)

In [9]:
# Testando a estrutura
descricao_empresa = "Uma empresa de tecnologia focada em inteligência artificial."
descricao_escola = "Uma escola voltada para educação infantil e fundamental."
descricao_outra_empresa = "Um novo produto para jardinagem."
descricao_outro = "Rosas são vermelhas, violetas são azuis"

In [10]:
# Chamadas de teste
print(multi_prompt_chain.invoke({"input":descricao_empresa}))

Com base nos nomes sugeridos, aqui estão algumas sugestões de slogans para uma campanha de propaganda:

**Para Aurora AI**:
* "Iluminando o futuro com inteligência artificial"
* "Despertando para uma nova era de conhecimento"
* "Aurora AI: onde a tecnologia encontra a inovação"

**Para CerebroX**:
* "Desbloqueando o potencial da inteligência artificial"
* "Pensando fora da caixa com CerebroX"
* "A inteligência artificial que você precisa para avançar"

**Para MindSpark**:
* "Faíscas de criatividade, chamas de inovação"
* "MindSpark: onde a inteligência artificial encontra a criatividade"
* "Despertando a mente para um futuro brilhante"

**Para Nexa Inteligência**:
* "Conectando você ao futuro da inteligência artificial"
* "Nexa Inteligência: a conexão que você precisa"
* "Inteligência artificial para um mundo conectado"

**Para Pulse AI**:
* "O pulso da inovação, a batida da tecnologia"
* "Pulse AI: onde a inteligência artificial encontra a energia"
* "Sentindo o pulso do futuro"

**Pa

In [11]:
print(multi_prompt_chain.invoke({"input":descricao_escola}))

Aqui estão algumas sugestões de slogans para uma campanha de propaganda para a escola:

1. **"Cresça com a gente!"** (para a Escola Crescer)
2. **"Aprenda, cresça, floresça!"** (para o Jardim de Aprendizado)
3. **"Educação que transforma vidas!"** (para a Escola Vida)
4. **"A felicidade começa aqui!"** (para a Criança Feliz)
5. **"Semeando conhecimento, colhendo sucesso!"** (para a Escola Semente)
6. **"Aprenda com alegria, cresça com confiança!"** (para o Aprendizado e Alegria)
7. **"Raízes fortes, futuro brilhante!"** (para a Escola Raízes)
8. **"O conhecimento é a chave para o sucesso!"** (para a Casa do Conhecimento)
9. **"Sonhe grande, alcance o impossível!"** (para a Escola Sonhos)
10. **"Jardim de sonhos, onde o futuro começa!"** (para o Jardim de Sonhos)

Esses slogans buscam ressaltar a importância da educação e do crescimento pessoal, além de destacar a missão e os valores da escola. Lembre-se de que o slogan ideal deve ser curto, memorável e transmitir a essência da escola.


In [12]:
print(multi_prompt_chain.invoke({"input":descricao_outra_empresa}))

Aqui estão algumas sugestões de slogans para uma campanha de propaganda para uma empresa de jardinagem com base nos nomes sugeridos anteriormente:

1. **VerdeVida**: "Viva a vida ao ar livre" ou "Transforme seu espaço em um oásis".
2. **JardimPlus**: "Mais beleza, mais vida" ou "O seu jardim, elevado ao próximo nível".
3. **Cultiva**: "Cultive a beleza" ou "Ajude a natureza a florescer".
4. **TerraVerde**: "Conecte-se com a natureza" ou "O seu jardim, um refúgio ecológico".
5. **Bloom**: "Faça com que o seu jardim floresça" ou "A beleza da natureza, ao seu alcance".
6. **Jardinagem Inteligente**: "Inove o seu jardim" ou "A tecnologia ao serviço da natureza".
7. **GreenCare**: "Cuide do seu jardim, cuide do planeta" ou "O seu jardim, um refúgio sustentável".
8. **Semente & Sol**: "Semeie a beleza" ou "Deixe o sol brilhar no seu jardim".
9. **Jardim Harmonia**: "Encontre a harmonia na natureza" ou "O seu jardim, um refúgio de paz".
10. **Cresça com Nós**: "Cresça com a natureza" ou "Junt

In [13]:
print(multi_prompt_chain.invoke({"input":descricao_outro}))

"Mas eu prefiro as rosas azuis."
